In [ ]:
import holoviews as hv
hv.notebook_extension('bokeh')

In [ ]:
# Info about RC visits

import re

visits = {'cosmos' : {'HSC-G' : "11690..11712:2^29324^29326^29336^29340^29350^29352",
                      'HSC-R' : "1202..1220:2^23692^23694^23704^23706^23716^23718",
                      'HSC-I' : "1228..1232:2^1236..1248:2^19658^19660^19662^19680^19682^19684^19694^19696^19698^19708^19710^19712^30482..30504:2",
                      'HSC-Y' : "274..302:2^306..334:2^342..370:2^1858..1862:2^1868..1882:2^11718..11742:2^22602..22608:2^22626..22632:2^22642..22648:2^22658..22664:2",
                      'HSC-Z' : "1166..1194:2^17900..17908:2^17926..17934:2^17944..17952:2^17962^28354..28402:2",
                      'NB0921' : "23038..23056:2^23594..23606:2^24298..24310:2^25810..25816:2"},
         'wide' : {'HSC-G' : "9852^9856^9860^9864^9868^9870^9888^9890^9898^9900^9904^9906^9912^11568^11572^11576^11582^11588^11590^11596^11598",
                   'HSC-R' : "11442^11446^11450^11470^11476^11478^11506^11508^11532^11534",
                   'HSC-I' : "7300^7304^7308^7318^7322^7338^7340^7344^7348^7358^7360^7374^7384^7386^19468^19470^19482^19484^19486",
                   'HSC-Y' : "6478^6482^6486^6496^6498^6522^6524^6528^6532^6544^6546^6568^13152^13154",
                   'HSC-Z' : "9708^9712^9716^9724^9726^9730^9732^9736^9740^9750^9752^9764^9772^9774^17738^17740^17750^17752^17754"}
         }
visits['wide-8766'] = visits['wide']
visits['wide-8767'] = visits['wide']

def visit_list(visit_string):
    l = []
    for v in visit_string.split('^'):
        try:
            l.append([int(v)])
        except:
            m = re.search('(\d+)\.\.(\d+):(\d)', v)
            l.append(range(int(m.group(1)), int(m.group(2))+1, int(m.group(3))))
    return [x for y in l for x in y]

def get_visits(field, filt):
    return visit_list(visits[field][filt])

def field_name(tract):
    if tract==9813:
        return 'cosmos'
    elif tract==8766:
        return 'wide-8766' 
    elif tract==8767:
        return 'wide-8767'

In [ ]:
from lsst.daf.persistence import Butler

rerun44 = '/project/tmorton/DM-12873/w44'
rerun46 = '/project/tmorton/DM-12873/w46'

butler44 = Butler(rerun44)
butler46 = Butler(rerun46)

butler = butler44

In [ ]:
from itertools import product
from functools import partial
import numpy as np

from lsst.pipe.analysis.utils import Filenamer

wide_filters = ['HSC-G', 'HSC-R', 'HSC-I', 'HSC-Z', 'HSC-Y']
cosmos_filters = ['HSC-G', 'HSC-R', 'HSC-I', 'HSC-Z', 'HSC-Y', 'NB0921']
COLUMNS = 3

def get_color_plot(butler, tract=8766, description='color_wPerp', style='psfMagHist', scale=1.0):
    dataId = {'tract':tract}
    filenamer = Filenamer(butler, 'plotColor', dataId)
    filename = filenamer(description=description, dataId=dataId, style=style)

    try:
        return hv.RGB.load_image(filename).opts(plot={'xaxis':None, 'yaxis':None,
                                                      'width':int(640*scale), 'height':int(480*scale)})
    except FileNotFoundError:
        return hv.RGB(np.zeros((2,2)))
    
def color_tract_layout(butler, description, style='psfMagHist', tracts=[8766, 8767, 9813], scale=1.0):
    return hv.Layout([get_color_plot(butler, tract, description=description, style=style, scale=scale) 
                         for tract in tracts])
    
def color_dmap(butler, tracts=[8766, 8767, 9813], descriptions=['color_wPerp', 'color_xPerp', 'color_yPerp'], 
               styles=['psfMagHist', 'sky-stars'], scale=1.0):
    
    dmap = hv.DynamicMap(partial(color_tract_layout, butler=butler, tracts=tracts, scale=scale), kdims=['description', 'style'])
    dmap = dmap.redim.values(description=descriptions, style=styles)
    return dmap

def get_plot(butler, tract, filt, description, style, visit=None, kind='coadd', scale=1.0):
    dataId = {'tract':tract, 'filter':filt}
    if visit is not None:
        dataId.update({'visit':visit})
        
    filenamer = Filenamer(butler, 'plot{}'.format(kind.capitalize()), dataId)
    filename = filenamer(description=description, style=style, dataId=dataId)
    try:
        return hv.RGB.load_image(filename).opts(plot={'xaxis':None, 'yaxis':None,
                                                      'width':int(640*scale), 'height':int(480*scale)})
    except FileNotFoundError:
        return hv.RGB(np.zeros((2,2)))
    
def filter_layout(butler, tract=9813, description='mag_modelfit_CModel', style='psfMagHist', visit=None, kind='coadd', scale=0.66):
    if tract==9813:
        filters = cosmos_filters
    else:
        filters = wide_filters
        
    return hv.Layout([get_plot(butler, tract, f, description, style, kind, scale=scale)
                               for f in filters]).cols(COLUMNS)
    
def description_layout(butler, descriptions, tract=9813, filt='HSC-I', style='psfMagHist', visit=None, kind='coadd', scale=0.66):
    return hv.Layout([get_plot(butler, tract, filt, desc, style, visit=visit, kind=kind, scale=scale) 
                               for desc in descriptions]).cols(COLUMNS)
    
def filter_layout_dmap_coadd(butler, descriptions, tracts=[8766, 8767, 9813], styles=['psfMagHist', 'sky-stars', 'sky-gals'], scale=0.66):
    dmap = hv.DynamicMap(partial(filter_layout, butler=butler, visit=None, kind='coadd', scale=scale), 
                     kdims=['tract', 'description', 'style'])
    dmap = dmap.redim.values(tract=tracts, description=descriptions, style=styles) 

    return dmap
    
def description_layout_dmap_visit(butler, tract, descriptions, filt='HSC-I', styles=['psfMagHist', 'sky-stars', 'sky-gals'], scale=0.66):
    visits = get_visits(field_name(tract), filt)
    dmap = hv.DynamicMap(partial(description_layout, descriptions=descriptions, butler=butler, tract=tract, filt=filt, kind='visit', scale=scale), 
                     kdims=['visit', 'style'])
    dmap = dmap.redim.values(visit=visits, style=styles)
    return dmap

# Source Catalog

## Aperture corrections for different photometry algorithms are consistent

In [ ]:
ap_descriptions = ['mag_base_GaussianFlux', 'mag_ext_photometryKron_KronFlux']
styles = ['psfMagHist', 'sky-stars', 'sky-gals']
filters = ['HSC-G', 'HSC-R', 'HSC-I', 'HSC-Z', 'HSC-Y']
tracts = [8766, 8767, 9813]

def visit_dmap_dict(tracts, filters, descriptions, styles=['psfMagHist', 'sky-stars', 'sky-gals'], **kwargs):
    return {tr : {f : description_layout_dmap_visit(butler, tr, descriptions=descriptions, filt=f, styles=styles, **kwargs).relabel('{}:{}'.format(tr, f)) for f in filters} for tr in tracts}
    

visit_dmap_aperture_wide = visit_dmap_dict([8766, 8767], wide_filters, ap_descriptions)
visit_dmap_aperture_cosmos = visit_dmap_dict([9813], cosmos_filters, ap_descriptions)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
for tr in [8766, 8767]:
    for f in wide_filters:
        visit_dmap_aperture_wide[tr][f]        

In [ ]:
for f in cosmos_filters:
    visit_dmap_aperture_cosmos[9813][f]

## PSF models correctly predict the ellipticities of stars over each tract.

In [ ]:
e_descriptions = ['e1ResidsHsm', 'e2ResidsHsm']

visit_dmap_quiver_wide = visit_dmap_dict([8766, 8767], wide_filters, descriptions=['ellipResids'], styles=['quiver'], scale=1.5)
visit_dmap_quiver_cosmos = visit_dmap_dict([9813], cosmos_filters, descriptions=['ellipResids'], styles=['quiver'], scale=1.5)

visit_dmap_ellip_wide = visit_dmap_dict([8766, 8767], wide_filters, e_descriptions, scale=1.)
visit_dmap_ellip_cosmos = visit_dmap_dict([9813], cosmos_filters, e_descriptions, scale=1.)

In [ ]:
%%opts RGB [height=600, width=800]

for tr in [8766, 8767]:
    for f in wide_filters:
        visit_dmap_quiver_wide[tr][f]     
        visit_dmap_ellip_wide[tr][f]       

In [ ]:
for f in cosmos_filters:
    visit_dmap_quiver_cosmos[9813][f]       
    visit_dmap_ellip_cosmos[9813][f]       

## Photometry measurements are consistent with reference catalog photometry (including sources not used in photometric calibration)

In [ ]:
ref_phot_descriptions=['matches_mag', 'matches_mag_calib_psf_used']
styles=['psfMagHist', 'sky-stars']

visit_dmap_ref_phot_wide = visit_dmap_dict([8766, 8767], wide_filters, ref_phot_descriptions, styles=styles, scale=1.)
visit_dmap_ref_phot_cosmos = visit_dmap_dict([9813], cosmos_filters, ref_phot_descriptions, styles=styles, scale=1.)

In [ ]:
for tr in [8766, 8767]:
    for f in wide_filters:
        visit_dmap_ref_phot_wide[tr][f]    

In [ ]:
for f in cosmos_filters:
    visit_dmap_ref_phot_cosmos[9813][f]       

## Astrometry measurements are consistent with reference catalog positions (including sources not used in astrometric calibration)

In [ ]:
ref_ast_descriptions=['matches_distance', 'matches_raCosDec', 'matches_dec']
styles=['psfMagHist', 'sky-stars']

visit_dmap_ref_ast_wide = visit_dmap_dict([8766, 8767], wide_filters, ref_ast_descriptions, styles=styles, scale=1.0)
visit_dmap_ref_ast_cosmos = visit_dmap_dict([9813], cosmos_filters, ref_ast_descriptions, styles=styles, scale=1.0)

In [ ]:
for tr in [8766, 8767]:
    for f in wide_filters:
        visit_dmap_ref_ast_wide[tr][f]    

In [ ]:
for f in cosmos_filters:
    visit_dmap_ref_ast_cosmos[9813][f]       

# Object Catalog

## Aperture corrections for different photometry algorithms are consistent

In [ ]:
filter_layout_dmap_coadd(butler, descriptions=['mag_modelfit_CModel', 'mag_base_GaussianFlux', 'mag_ext_photometryKron_KronFlux'], scale=1.)

## PSF models correctly predict the ellipticities of stars over the field of view.

In [ ]:
filter_layout_dmap_coadd(butler, descriptions=['e1ResidsHsm', 'e2ResidsHsm'], styles=['psfMagHist'], scale=1.)

In [ ]:
filter_layout_dmap_coadd(butler, descriptions=['ellipResids'], styles=['quiver'], scale=1.)

## Photometry measurements are consistent with reference catalog photometry (including sources not used in photometric calibration)

In [ ]:
filter_layout_dmap_coadd(butler, descriptions=['matches_mag', 'matches_mag_calib_psf_used'], styles=['psfMagHist', 'sky-stars'], scale=1.)

## Astrometry measurements are consistent with reference catalog positions (including sources not used in astrometric calibration)

In [ ]:
filter_layout_dmap_coadd(butler, descriptions=['matches_distance', 'matches_raCosDec', 'matches_dec'], styles=['psfMagHist', 'sky-stars'], scale=1)

## Forced and unforced photometry measurements are consistent.

In [ ]:
filter_layout_dmap_coadd(butler, descriptions=['compareUnforced_modelfit_CModel'], scale=1.)

## The slope of the stellar locus in color-color space is not a function of position on the sky

In [ ]:
color_dmap(butler, descriptions=['color_wPerp', 'color_xPerp', 'color_yPerp'])

In [ ]:
color_dmap(butler, descriptions=['gri', 'izy', 'riz'], styles=['fit', 'psfMagHist', 'sky-stars'])